# Calculate the water volume on the pond

In [1]:
%matplotlib widget
import numpy as np
import cv2
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import mpl_toolkits.mplot3d.art3d as art3d

In [2]:
# read the image, location of measurement points marked in gimp
img = cv2.imread('./gmap-points.png')[:,:,::-1]
plt.figure(figsize=(6,6))
plt.imshow(img)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
height = img.shape[0]
width = img.shape[1]

In [4]:
def exact_color_mask(img, color):
    """Return a mask when img matches the exact color given."""
    mask = np.zeros((img.shape[0], img.shape[1]), np.bool)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            pixel = img[i,j]
            if (pixel == color).all():
                mask[i,j] = True
    return mask.astype(int)

def exact_color_mask_fast(img, color):
    """Return a mask when img matches the exact color given."""
    return np.all(np.equal(img[:,:], color),axis=2).astype(np.uint8)

# red = exact_color_mask(img, (255, 0, 0))
# blue = exact_color(img, (0, 0, 255))
red = exact_color_mask_fast(img, (255, 0, 0))
blue = exact_color_mask_fast(img, (0, 0, 255))

In [5]:
plt.figure()
plt.subplot(121)
plt.imshow(blue,'gray')
plt.subplot(122)
plt.imshow(red,'gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
def find_center_of_contour(contour):
        M = cv2.moments(contour)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        return cX, cY
    
def find_color_dot_coords(image):
    """Returns a list of (y,x) pairs of dots of color found in image"""
    contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return [find_center_of_contour(c) for c in contours]    

In [7]:
blue_points = find_color_dot_coords(blue)
red_points = find_color_dot_coords(red)
red_points

[(1403, 2995),
 (1039, 2987),
 (1695, 2927),
 (1407, 2695),
 (1120, 2696),
 (1699, 2635),
 (940, 2575),
 (1112, 2438),
 (1442, 2360),
 (1863, 2303),
 (939, 2059),
 (1123, 2023),
 (1839, 2015),
 (1283, 1995),
 (1475, 1983),
 (914, 1727),
 (1170, 1712),
 (1915, 1707),
 (1583, 1547),
 (1387, 1547),
 (1979, 1403),
 (1815, 1307),
 (895, 1303),
 (1507, 1099),
 (1311, 999)]

In [8]:
# blue points + 0 (ground level)
ground_level = [(x, y, 0) for x,y in blue_points]

# red_points
# x,y copy pasted from red_points
# depth typed in by hand
measurements = [ (1403, 2995, 100),
                 (1039, 2987, 90),
                 (1695, 2927, 100),
                 (1407, 2695, 130),
                 (1120, 2696, 130),
                 (1699, 2635, 130),
                 (940, 2575, 120),
                 (1112, 2438, 140),
                 (1442, 2360, 160),
                 (1863, 2303, 150),
                 (939, 2059, 150),
                 (1123, 2023, 190),
                 (1839, 2015, 180),
                 (1283, 1995, 200),
                 (1475, 1983, 220),
                 (914, 1727, 150),
                 (1170, 1712, 200),
                 (1915, 1707, 230),
                 (1583, 1547, 240),
                 (1387, 1547, 230),
                 (1979, 1403, 150),
                 (1815, 1307, 180),
                 (895, 1303, 150),
                 (1507, 1099, 210),
                 (1311, 999, 190)]

points = np.array(ground_level + measurements, dtype=float)

# pixels to meters
points /= 100  
height = img.shape[0] / 100
width = img.shape[1] / 100
measurements = np.array(measurements, dtype=float) / 100

In [9]:
res = 4
grid_x, grid_y = np.mgrid[0:width:width*res*1j, 0:height:height*res*1j]

In [10]:
method = 'cubic'
grid_z = griddata(points[:,:2], points[:,2], (grid_x, grid_y), method=method)
grid_z = np.nan_to_num(grid_z).clip(min=0)

In [11]:
width, height, res

(29.0, 38.0, 4)

In [12]:
plt.figure(figsize=(6,6))
plt.imshow(-grid_z.T, cmap='winter')
plt.colorbar();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
fig = plt.figure(figsize=(8,8))
ax = fig.gca(projection='3d')
ax.scatter(grid_x.reshape(-1), -grid_y.reshape(-1), -grid_z.reshape(-1), c=-grid_z.reshape(-1), marker='.', cmap='winter')
for xi, yi, zi in zip(measurements[:,0], -measurements[:,1], -measurements[:,2]):        
    line=art3d.Line3D(*zip((xi, yi, 0), (xi, yi, zi)), c='r', marker='o', markevery=(1, 1))
    ax.add_line(line)
ax.auto_scale_xyz([0, 40], [-40, 0], [-10, 10])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
volume = np.sum(grid_z)/res**2
print(f"Water volume = {volume:.0f}m^3.")
print(f"Interpolated at {1/res}m step grid")

Water volume = 449m^3.
Interpolated at 0.25m step grid


# Pond Area

In [15]:
# read in image, area painted in gimp
img = cv2.imread('./gmap-area.png')[:,:,::-1]
plt.figure(figsize=(6,6))
plt.imshow(img)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
%%time
# TODO: optimize this cell, too slow
height = img.shape[0]
width = img.shape[1]
red = exact_color_mask_fast(img, (255, 0, 0))

CPU times: user 249 ms, sys: 5.67 ms, total: 254 ms
Wall time: 254 ms


In [17]:
area = np.sum(red) / 100 / 100
print(f"Water area = {area:.0f}m^2")
print(f"Average depth = {volume/area:.1f}m")

Water area = 372m^2
Average depth = 1.2m
